In [1]:
import collections
import sys
import argparse
import numpy as np
import sklearn as sk
import scipy as sp
import scipy.optimize
import glob
import scipy.stats
import imageio
import math
from camera_pipeline_wb import after_wb
from PIL import Image
import torch
import pandas as pd
from datetime import datetime
import rawpy
import matplotlib
from tqdm.notebook import tqdm
import cv2
from matplotlib import pyplot as plt
from skimage import exposure
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle, estimate_sigma
from skimage.morphology import closing, opening, erosion, dilation, disk, diamond, square
torch.set_default_dtype(torch.float32)

In [2]:
def process_image(filepath):
  ''' Gets image from filepath.performs linearization and demosaicing'''
  def get_channels(bayer_image):
    blue = bayer_image[1::2,1::2]
    red = bayer_image[0::2,0::2]
    green1 = bayer_image[1::2,0::2]
    green2 = bayer_image[0::2,1::2]
    green = (green1 + green2)/2
    return red, green, blue
  
  #Loading the RGB image
  with rawpy.imread(filepath) as raw1:

      imageio.imwrite("og_image.jpg",raw1.postprocess())
      rgb=raw1.raw_image_visible
      saturation=raw1.camera_white_level_per_channel[0]
      black=min(raw1.black_level_per_channel[0],rgb.min())
      rgb=rgb.astype(np.float32)
      rgb_linearized=(rgb-black)/(saturation-black)
      rgb1=get_channels(rgb_linearized)
      rgb_image=np.dstack(rgb1)
      raw1.close()
 
  return np.array(rgb_image)


def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):

  dim = None
  (h, w) = image.shape[:2]  
  if width is None and height is None:
      return image
  if width is not None and height is not None:
    return cv2.resize(image,(width,height),interpolation=inter)
  if width is None:
      r = height / float(h)
      dim = (int(w * r), height)
  else:
      r = width / float(w)
      dim = (width, int(h * r))

  resized = cv2.resize(image, dim, interpolation = inter)
  return resized



def scale(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img))

def preprocess_sfm_depth_map(depths, min_depth, max_depth):
    z_min = np.min(depths) + (min_depth * (np.max(depths) - np.min(depths)))
    z_max = np.min(depths) + (max_depth * (np.max(depths) - np.min(depths)))
    if max_depth != 0:
        depths[depths == 0] = z_max

    return depths

In [3]:
"""
Backscatter and Preprocessing Path
"""
size = 1024;f=2;l=0.5;p=0.002;min_depth=0.1;max_depth=1.0;fraction = 0.01

def load_image_and_depth_map(img_fname, depths_fname, size_limit = 512):
    
    depths = np.array(Image.open(depths_fname))
    img  = process_image(img_fname)
    resized_img = image_resize(img,width=size_limit)
    depth_resized = image_resize(depths,height=resized_img.shape[0],width=size_limit)
    return resized_img, depth_resized


def find_backscatter_estimation_points(img, depths, num_bins=10, fraction=0.01, max_vals=100, min_depth_percent=0.0):
    z_max, z_min = np.max(depths), np.min(depths)
    min_depth = z_min + (min_depth_percent * (z_max - z_min))
    z_ranges = np.linspace(z_min, z_max, num_bins + 1)
    img_norms = np.mean(img, axis=2)
    #img_norms = 0.2989 * img[:,:,0] + 0.5870 * img[:,:,1] + 0.1140 * img[:,:,2] #Change here
    points_r = []
    points_g = []
    points_b = []
    for i in range(len(z_ranges) - 1):
        a, b = z_ranges[i], z_ranges[i+1]
        locs = np.where(np.logical_and(depths > min_depth, np.logical_and(depths >= a, depths <= b)))
        norms_in_range, px_in_range, depths_in_range = img_norms[locs], img[locs], depths[locs]
        arr = sorted(zip(norms_in_range, px_in_range, depths_in_range), key=lambda x: x[0])
        points = arr[:min(math.ceil(fraction * len(arr)), max_vals)]
        points_r.extend([(z, p[0]) for n, p, z in points])
        points_g.extend([(z, p[1]) for n, p, z in points])
        points_b.extend([(z, p[2]) for n, p, z in points])

    return np.array(points_r), np.array(points_g), np.array(points_b)

def find_backscatter_values(B_pts, depths, restarts=100, max_mean_loss_fraction=0.1):
    B_vals, B_depths = B_pts[:, 1], B_pts[:, 0]
    z_max, z_min = np.max(depths), np.min(depths)
    max_mean_loss = max_mean_loss_fraction * (z_max - z_min)
    coefs = None
    best_loss = np.inf

    def estimate(depths, B_inf, beta_B, J_prime, beta_D_prime):
        val = (B_inf * (1 - np.exp(-1 * beta_B * depths))) + (J_prime * np.exp(-1 * beta_D_prime * depths))
        return val
    def loss(B_inf, beta_B, J_prime, beta_D_prime):
        val = np.mean(np.abs(B_vals - estimate(B_depths, B_inf, beta_B, J_prime, beta_D_prime)))
        return val
    
    bounds_lower = [0,0,0,0]
    bounds_upper = [1,5,1,5]
    
    for _ in range(restarts):
        try:
            optp, pcov = sp.optimize.curve_fit(
                f=estimate,
                xdata=B_depths,
                ydata=B_vals,
                p0=np.random.random(4) * bounds_upper,
                bounds=(bounds_lower, bounds_upper),
            )
            l = loss(*optp)
            if l < best_loss:
                best_loss = l
                coefs = optp
        except RuntimeError as re:
            print(re, file=sys.stderr)
    if best_loss > max_mean_loss:
        print('Warning: could not find accurate reconstruction. Switching to linear model.', flush=True)
        slope, intercept, r_value, p_value, std_err = sp.stats.linregress(B_depths, B_vals)
        BD = (slope * depths) + intercept
        return BD, np.array([slope, intercept])

    print(best_loss)
    return estimate(depths, *coefs), coefs

In [4]:
def estimate_wideband_attentuation(depths, illum, radius = 6, max_val = 10.0):
      eps = 1E-8
      BD = np.minimum(max_val, -np.log(illum + eps) / (np.maximum(0, depths) + eps))
      mask = np.where(np.logical_and(depths > eps, illum > eps), 1, 0)
      refined_attenuations = denoise_bilateral(closing(np.maximum(0, BD * mask), disk(radius)))
      return refined_attenuations, []

In [5]:
def filter_data(X, Y, radius_fraction=0.01):
    idxs = np.argsort(X)
    print(idxs)
    idxs = idxs.astype(np.int32)
    X_s = X[idxs]
    idxs = idxs.astype(np.int32)
    Y_s = Y[idxs]
    x_max, x_min = np.max(X), np.min(X)
    radius = (radius_fraction * (x_max - x_min))
    ds = np.cumsum(X_s - np.roll(X_s, (1,)))
    dX = [X_s[0]]
    dY = [Y_s[0]]
    tempX = []
    tempY = []
    pos = 0
    for i in range(1, ds.shape[0]):
        if ds[i] - ds[pos] >= radius:
            tempX.append(X_s[i])
            tempY.append(Y_s[i])
            idxs = np.argsort(tempY)
            med_idx = len(idxs) // 2
            dX.append(tempX[med_idx])
            dY.append(tempY[med_idx])
            pos = i
        else:
            tempX.append(X_s[i])
            tempY.append(Y_s[i])
    return np.array(dX), np.array(dY)

In [6]:

'''

Estimate coefficients for the 2-term exponential
describing the wideband attenuation
'''
def refine_wideband_attentuation(depths, illum, estimation, restarts=25, min_depth_fraction = 0.1, max_mean_loss_fraction=np.inf, l=1.0, radius_fraction=0.01):
    eps = 1E-8
    z_max, z_min = np.max(depths), np.min(depths)
    min_depth = z_min + (min_depth_fraction * (z_max - z_min))
    max_mean_loss = max_mean_loss_fraction * (z_max - z_min)
    coefs = None
    best_loss = np.inf
    locs = np.where(np.logical_and(illum > 0, np.logical_and(depths > min_depth, estimation > eps)))
    def calculate_reconstructed_depths(depths, illum, a, b, c, d):
        eps = 1E-5
        res = -np.log(illum + eps) / (calculate_beta_D(depths, a, b, c, d) + eps)
        return res
    def loss(a, b, c, d):
        return np.mean(np.abs(depths[locs] - calculate_reconstructed_depths(depths[locs], illum[locs], a, b, c, d)))
    dX, dY = filter_data(depths[locs], estimation[locs], radius_fraction)
    for _ in range(restarts):
        try:
            optp, pcov = sp.optimize.curve_fit(
                f=calculate_beta_D,
                xdata=dX,
                ydata=dY,
                p0=np.abs(np.random.random(4)) * np.array([1., -1., 1., -1.]),
                bounds=([0, -100, 0, -100], [100, 0, 100, 0]))
            L = loss(*optp)
            if L < best_loss:
                best_loss = L
                coefs = optp
        except RuntimeError as re:
            print(re, file=sys.stderr)
    # Uncomment to see the regression
    # plt.clf()
    # plt.scatter(depths[locs], estimation[locs])
    # plt.plot(np.sort(depths[locs]), calculate_beta_D(np.sort(depths[locs]), *coefs))
    # plt.show()
    print(f'Found best loss {best_loss}', flush=True)
    BD = l * calculate_beta_D(depths, *coefs)
    return BD, coefs,best_loss

  
def wbalance_no_red_10p(img):
    dg = 1.0 / np.mean(np.sort(img[:, :, 1], axis=None)[int(round(-1 * np.size(img[:, :, 0]) * 0.1)):])
    db = 1.0 / np.mean(np.sort(img[:, :, 2], axis=None)[int(round(-1 * np.size(img[:, :, 0]) * 0.1)):])
    dsum = dg + db
    dg = dg / dsum * 2.
    db = db / dsum * 2.
    img[:, :, 0] *= (db + dg) / 2
    img[:, :, 1] *= dg
    img[:, :, 2] *= db
    return img
    
def recover_image(img, depths, B, beta_D):
    
    res = (img - B) * np.exp(beta_D * np.expand_dims(depths, axis=2))
    print(res.max(),res.min())
    print(np.percentile(res, range(0,100,5)))
    res = np.maximum(0.0, np.minimum(1.0, res))
    res = scale(wbalance_no_red_10p(res))
    return res


In [7]:
'''
Estimate coefficients for the 2-term exponential
describing the wideband attenuation
'''
'''
Calculate the values of beta_D for an image from the depths, illuminations, and constants
'''
def calculate_beta_D(depths, a,b,c,d):
    return a * np.exp(-b * depths)+ c * np.exp(-d * depths)


def refine_wideband_attentuation_neham(depths, illum, estimation, restarts=10, min_depth_fraction = 0.1, max_mean_loss_fraction=np.inf, l=1.0, radius_fraction=0.01):
    eps = 1E-8
    min_depth = np.percentile(depths, 10, axis=None, out=None) 
    max_depth = np.percentile(depths, 90, axis=None, out=None) 
    best_loss = np.inf

    pts_to_consider = np.where(np.logical_and(depths>=min_depth,depths<=max_depth))
    #print(pts_to_consider)
    dX = depths[pts_to_consider]
    dY = depths[pts_to_consider]

    def calculate_reconstructed_depths(depths, illum, a, b,c,d):
        eps = 1E-5
        res = -np.log(illum + eps) / (calculate_beta_D(depths, a, b,c,d) + eps)
        return res
    
    def loss(a, b,c,d):
        return np.mean(np.abs(depths[pts_to_consider] - calculate_reconstructed_depths(depths[pts_to_consider], illum[pts_to_consider], a, b,c,d)))
    
    
    for _ in range(restarts):
        try:
            optp, pcov = sp.optimize.curve_fit(
                f=calculate_beta_D,
                xdata=dX,
                ydata=dY,
                p0=np.abs(np.random.random(4)) * np.array([5., 3.,4,5]),
                bounds=([0, 0,0,0], [100, 100,100,100]))
            L = loss(*optp)
            if L < best_loss:
                best_loss = L
                coefs = optp
        except RuntimeError as re:
            print(re, file=sys.stderr)
    # Uncomment to see the regression
    # plt.clf()
    # plt.scatter(depths[locs], estimation[locs])
    # plt.plot(np.sort(depths[locs]), calculate_beta_D(np.sort(depths[locs]), *coefs))
    # plt.show()
    print(f'Found best loss {best_loss}', flush=True)
    BD = l * calculate_beta_D(depths, *coefs)
    return BD, coefs,best_loss

In [8]:
def depth_iterative(without_backscatter,pd,normalized_depth,count,epochs):
  dc=torch.zeros((without_backscatter.shape[0],without_backscatter.shape[1])).cuda()
  pd=torch.tensor(pd, dtype=torch.float32).cuda()
  normalized_depth=torch.from_numpy(normalized_depth).cuda()
  count=torch.from_numpy(count).cuda()

  for i in tqdm(range(epochs)):

    dc_temp=torch.zeros((without_backscatter.shape[0],without_backscatter.shape[1])).cuda()
    #Left Condition
    dc_temp[:,1:] = dc_temp[:,1:] + dc[:,:-1]
    #Right Condition
    dc_temp[:,:-1] = dc_temp[:,:-1] + dc[:,1:]
    #Top Condition
    dc_temp[1:,:] = dc_temp[1:,:] + dc[:-1,:]
    #Bottom Condition
    dc_temp[:-1,:] = dc_temp[:-1,:] + dc[1:,:]

    dc_temp=torch.div(dc_temp,count)

    dc = pd*normalized_depth + (1-pd)* dc_temp

    if i%5000==0:
      print("Epoch: ",i,"Mean: ",dc.mean())
    
  return dc

In [9]:
def determine_depth_threshold(without_backscatter,normalized_depth,depth_image,pd,epochs=25000):
  
  count=4*np.ones(((without_backscatter.shape[0],without_backscatter.shape[1])))
  count[0,:]=count[0,:]-1
  count[-1,:]=count[-1,:]-1
  count[:,0]=count[:,0]-1
  count[:,-1]=count[:,-1  ]-1
  count=count.astype(np.float32)
  dc=depth_iterative(without_backscatter,pd,normalized_depth,count,epochs)
  return dc

In [10]:
def iterative_func(count,without_backscatter,lc,rc,tc,bc,epochs,p):
  p=torch.tensor(p, dtype=torch.float32).cuda()
  lc=torch.from_numpy(lc).cuda()
  rc=torch.from_numpy(rc).cuda()
  tc=torch.from_numpy(tc).cuda()
  bc=torch.from_numpy(bc).cuda()
  count=torch.from_numpy(count).cuda()

  ac=torch.zeros((without_backscatter.shape[0],without_backscatter.shape[1],without_backscatter.shape[2])).cuda()
  without_backscatter=torch.from_numpy(without_backscatter).cuda()
  for i in tqdm(range(epochs)):
    #temp1=compute_ac_dash(temp,lc,rc,tc,bc,ac)
    #ac_temp = np.divide(compute_ac_dash(temp,lc,rc,tc,bc,ac), count, out=np.zeros_like(count), where=count!=0)
    ac_temp=torch.zeros((without_backscatter.shape[0],without_backscatter.shape[1],without_backscatter.shape[2])).cuda()
    #Left Condition
    ac_temp[:,1:] = ac_temp[:,1:] + torch.mul(lc[:,1:,:],ac[:,:-1,:])    

    #Right Condition
    ac_temp[:,:-1] = ac_temp[:,:-1] + torch.mul(rc[:,:-1],ac[:,1:])
    #Top Condition
    ac_temp[1:,:] = ac_temp[1:,:] + torch.mul(tc[1:,:],ac[:-1,:])
    #Bottom Condition
    ac_temp[:-1,:] = ac_temp[:-1,:] + torch.mul(bc[:-1,:],ac[1:,:])
    
    ac_temp=torch.div(ac_temp,count)

    ac_temp[torch.isnan(ac_temp)] = 0 
       
    ac = p*without_backscatter + (1-p)* ac_temp
    if i%5000==0:
      print("Epoch: ",i,"Mean: ",ac.mean())
    
  
  return ac
  

In [11]:
def LSAC(without_backscatter,normalized_depth,threshold,p,epochs=25000):
  
  def determine_neighbourhood(depth_image,threshold):
    #Check Left Condition 
    lc=abs(depth_image[:,1:]-depth_image[:,:-1])<threshold[:,1:]
    lc=np.hstack((np.zeros((lc.shape[0],1)),lc))
    lc=lc[:,:,np.newaxis]
    #Check Right Condition 
    rc=abs(depth_image[:,:-1]-depth_image[:,1:])<threshold[:,:-1]
    rc=np.hstack((rc,np.zeros((rc.shape[0],1))))
    rc=rc[:,:,np.newaxis]
    
    #Check Top Condition 
    tc=abs(depth_image[1:,:]-depth_image[:-1,:])<threshold[1:,:]
    tc=np.vstack((np.zeros((1,tc.shape[1])),tc))
    tc=tc[:,:,np.newaxis]

    #Check Bottom Condition 
    bc=abs(depth_image[:-1,:]-depth_image[1:,:])<threshold[:-1,:]
    bc=np.vstack((bc,np.zeros((1,bc.shape[1]))))
    bc=bc[:,:,np.newaxis]
    lc=lc.astype(np.float32)
    rc=rc.astype(np.float32)
    tc=tc.astype(np.float32)
    bc=bc.astype(np.float32)

    return lc,rc,tc,bc
  

  lc,rc,tc,bc=determine_neighbourhood(normalized_depth,threshold=threshold)
  count=lc+rc+tc+bc
  without_backscatter=without_backscatter.astype(np.float32)
  ac=iterative_func(count,without_backscatter,lc,rc,tc,bc,epochs,p)    
  return ac


In [12]:
size = 1024;f=2;l=0.5;p=0.002;min_depth=0.1;max_depth=1.0;fraction = 0.01

def seathru(img_path,depth_path,save_path):

    raw = rawpy.imread(img_path)
    img, depths = load_image_and_depth_map(img_path,depth_path)
    save_coeffs = {}
    depths = preprocess_sfm_depth_map(depths,min_depth,max_depth)
    print('Estimating backscatter...', flush=True)
    ptsR, ptsG, ptsB = find_backscatter_estimation_points(img, depths, fraction=0.01, min_depth_percent=min_depth)

    print('Finding backscatter coefficients...', flush=True)
    Br, coefsR = find_backscatter_values(ptsR, depths, restarts=100)
    Bg, coefsG = find_backscatter_values(ptsG, depths, restarts=100)
    Bb, coefsB = find_backscatter_values(ptsB, depths, restarts=100)
    B = np.stack([Br, Bg, Bb], axis=2)
    
    without_backscatter = np.clip(img-B,0,1)
    print('Estimating wideband attenuation...', flush=True)

    pc=5e-4
    pd=5e-4

    normalized_depth = (depths-np.min(depths))/(np.max(depths)-np.min(depths))

    dc=determine_depth_threshold(without_backscatter.copy(),normalized_depth.copy(),depths.copy(),pd,epochs=25000)
    dc=dc.cpu().detach().numpy()  

    #Part 4
    threshold=0.1*dc

    ac=LSAC(without_backscatter,normalized_depth,threshold,pd,epochs=25000)
    ac=ac.cpu().detach().numpy()  
    illumination=2*ac
    illumination[np.where(illumination==0)] = illumination[np.where(illumination==0)]+1e-5

    illR = illumination[:,:,0]
    illG = illumination[:,:,1]
    illB = illumination[:,:,2]

    beta_D_r, _ = estimate_wideband_attentuation(depths, illR)
    #refined_beta_D_r, coefsR,best_loss_R = refine_wideband_attentuation_neham(depths, illR, beta_D_r, radius_fraction = fraction, l=l)

    beta_D_g, _ = estimate_wideband_attentuation(depths, illG)
    #refined_beta_D_g, coefsG,best_loss_G = refine_wideband_attentuation_neham(depths, illG, beta_D_g, radius_fraction = fraction, l=l)

    beta_D_b, _ = estimate_wideband_attentuation(depths, illB)
    #refined_beta_D_b, coefsB,best_loss_B = refine_wideband_attentuation_neham(depths, illB, beta_D_b, radius_fraction = fraction, l=l)

    save_coeffs["file_name"] = img_path 
    save_coeffs["depth_mean"] = depths.mean()

    print('Reconstructing image...', flush=True)
    B = np.stack([Br, Bg, Bb], axis=2)
    beta_D_og = np.stack([beta_D_r,beta_D_g,beta_D_b], axis=2)
    #refined_beta_D_og = np.stack([refined_beta_D_r,refined_beta_D_g,refined_beta_D_b], axis=2)

    recovered_og = recover_image(img, depths, B, beta_D_og)

    #recovered_refined = recover_image(img, depths, B, refined_beta_D_og)
    
    imageio.imwrite(save_path,recovered_og)
    imageio.imwrite("results/"+"postprocessed"+save_path.split('/')[-1],raw.postprocess(half_size=True))
    after_wb(raw,img,"results/"+"original"+save_path.split('/')[-1])
    return np.exp(beta_D_og * np.expand_dims(depths, axis=2)),depths

In [13]:
def beta_D(coefs,depths):
  return coefs[0]*np.exp(-coefs[1]*depths) + coefs[2]*np.exp(-coefs[3]*depths)

In [14]:
files = glob.glob("/home/neham/uw_datasets/SeaThru_Final/*")
input_files = []
input_depths = []
outputs = []
count = 0
for i in files:
    raw_files = glob.glob(i+"/Raw/*")[-2:]
    list_depth = glob.glob(i+"/depthMaps/*")
    for file_name in raw_files:
        depth_num = file_name.split(".")[0][-3:]
        depth_file = [i for i in list_depth if depth_num in i]
        if len(depth_file)==0:
          continue
        file_d = depth_file[0]
        output_name = "results/" + file_name.split("/")[-1][:-3] + "png"
        count+=1
        input_files.append(file_name)
        input_depths.append(file_d)
        outputs.append(output_name)
  

In [16]:
input_files

['/home/neham/uw_datasets/SeaThru_Final/SeaThru_8/Raw/T_S04863.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_8/Raw/T_S04895.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_4/Raw/T_S03507.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_4/Raw/T_S03488.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_11/Raw/LFT_3414.NEF',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_11/Raw/LFT_3413.NEF',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_3/Raw/T_S03386.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_3/Raw/T_S03422.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_0/Raw/T_S03132.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_0/Raw/T_S03107.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_9/Raw/RGT_0217.NEF',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_9/Raw/RGT_0178.NEF',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_2/Raw/T_S03336.ARW',
 '/home/neham/uw_datasets/SeaThru_Final/SeaThru_2/Raw/T_S03282.ARW',
 '/home/neham/uw_datasets/SeaThr

In [15]:
for i,j,k in tqdm(zip(input_files,input_depths,outputs)):
    try:
        direct_attenuation,depths = seathru(i,j,k)
        break
    except:
        pass

0it [00:00, ?it/s]

Estimating backscatter...
Finding backscatter coefficients...
0.0010819504
0.0050101727
0.0033833594
Estimating wideband attenuation...


  0%|          | 0/25000 [00:00<?, ?it/s]

Epoch:  0 Mean:  tensor(0.0003, device='cuda:0')
Epoch:  5000 Mean:  tensor(0.5438, device='cuda:0')
Epoch:  10000 Mean:  tensor(0.5884, device='cuda:0')
Epoch:  15000 Mean:  tensor(0.5920, device='cuda:0')
Epoch:  20000 Mean:  tensor(0.5923, device='cuda:0')


  0%|          | 0/25000 [00:00<?, ?it/s]

Epoch:  0 Mean:  tensor(2.2894e-05, device='cuda:0')
Epoch:  5000 Mean:  tensor(0.0418, device='cuda:0')
Epoch:  10000 Mean:  tensor(0.0452, device='cuda:0')
Epoch:  15000 Mean:  tensor(0.0455, device='cuda:0')
Epoch:  20000 Mean:  tensor(0.0455, device='cuda:0')
Reconstructing image...


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


596372043037.5481 -6675015257.106591
[-6.67501526e+09  1.65949170e-02  8.13733900e-02  1.36490499e-01
  1.94589236e-01  2.59015039e-01  3.24243811e-01  3.83502841e-01
  4.31728178e-01  4.66206202e-01  4.86514227e-01  4.98668791e-01
  5.07333012e-01  5.17404253e-01  5.45407583e-01  6.03395343e-01
  6.71001404e-01  7.49311505e-01  8.49061672e-01  1.02834293e+00]


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [15]:
for i in input_depths:
    depths = np.array(Image.open(i))
    print(depths.max(),depths.min(),i)

2.9585354 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_8/depthMaps/depthT_S04863.tif
2.8167076 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_8/depthMaps/depthT_S04895.tif
0.97354454 0.4955313 /home/neham/uw_datasets/SeaThru_Final/SeaThru_4/depthMaps/depthT_S03507.tif
1.2640331 0.6778563 /home/neham/uw_datasets/SeaThru_Final/SeaThru_4/depthMaps/depthT_S03488.tif
4.4719105 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_11/depthMaps/depthLFT_3414.tif
5.2783165 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_11/depthMaps/depthLFT_3413.tif
1.2959849 0.6499919 /home/neham/uw_datasets/SeaThru_Final/SeaThru_3/depthMaps/depthT_S03386.tif
1.1321101 0.6963256 /home/neham/uw_datasets/SeaThru_Final/SeaThru_3/depthMaps/depthT_S03422.tif
1.817156 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_0/depthMaps/depthT_S03132.tiff
2.0637763 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_0/depthMaps/depthT_S03107.tiff
2.998172 0.0 /home/neham/uw_datasets/SeaThru_Final/SeaThru_9/depthMaps/d